In [34]:
import random

class Sentiment:
    NEGATIVE = "Negative"
    NEUTRAL = "Neutral"
    POSITIVE = "Positive"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score 
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: 
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return[x.text for x in self.reviews]
    
    def get_sentiment(self):
        return[x.sentiment for x in self.reviews]
        
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment. NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment. POSITIVE, self.reviews))

        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk 
        random.shuffle(self.reviews)



In [35]:
import json

file_name = "true.txt"

reviews = []
with open(file_name) as f: 
    for lines in f:
        review = json.loads(lines)
        reviews.append((Review(review['reviewText'], review['overall'])))

reviews[100].sentiment

'Positive'

In [46]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size= 0.33, random_state= 42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)


In [52]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_x = test_container.get_text()
test_y = test_container.get_sentiment()


print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

It was just one of those books that never went anywhere. I like books that get your attention in the beginning and not drag out until a quarter way through. I decided to give it an early death - delete!
436


AttributeError: 'ReviewContainer' object has no attribute 'get_text'

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit_transform(train_x)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 428424 stored elements and shape (6700, 26615)>

In [24]:
#Bags of words vectorization

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vector = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vector[0].toarray())
        

Olivia Hampton arrives at the Dunraven family home as cataloger of their extensive library. What she doesn't expect is a broken carriage wheel on the way. Nor a young girl whose mind is clearly gone, an old man in need of care himself (and doesn&#8217;t quite seem all there in Olivia&#8217;s opinion). Furthermore, Marion Dunraven, the only sane one of the bunch and the one Olivia is inexplicable drawn to, seems captive to everyone in the dusty old house. More importantly, she doesn't expect to fall in love with Dunraven's daughter Marion.Can Olivia truly believe the stories of sadness and death that surround the house, or are they all just local neighborhood rumor?Was that carriage trouble just a coincidence or a supernatural sign to stay away? If she remains, will the Castle&#8217;s dark shadows take Olivia down with them or will she and Marion long enough to declare their love?Patty G. Henderson has created an atmospheric and intriguing story in her Gothic tale. I found this to be an

In [53]:
#Classification
#Linear SVM

from sklearn import svm

clf_svm = svm.SVC(kernel= "linear")
clf_svm.fit(train_x_vector, train_y)

test_x[0]
test_x_vectors[0]

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 91 stored elements and shape (1, 26615)>

In [54]:
#Decision Tree

from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vector, train_y)

clf_dec.predict(test_x_vectors[0])


array(['Positive'], dtype='<U8')

In [57]:
#Naive Bayes

from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vector, train_y)

clf_dec.predict(test_x_vectors[0])


TypeError: Sparse data was passed for X, but dense data is required. Use '.toarray()' to convert to a dense numpy array.

In [56]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vector, train_y)

clf_log.predict(test_x_vectors[0])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['Positive'], dtype='<U8')

In [60]:
#Evaluation

print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))



0.8124242424242424
0.7712121212121212


TypeError: Sparse data was passed for X, but dense data is required. Use '.toarray()' to convert to a dense numpy array.

In [61]:
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, 
                                                                        Sentiment.NEGATIVE])
f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, 
                                                                        Sentiment.NEGATIVE])

array([0.92139968, 0.29250457, 0.40983607])

In [63]:
train_y.count(Sentiment.POSITIVE)

5611